In [ ]:
#host:  aislab.hpc.di.unimi.it
#user: bonomi
#psw: capsnetwork

# prova minst su affnist e verificare risultati con paper
# testare su dataset normale + rotraslato*2
# risultato si su db normale sia rototraslato
#
# + provare no shearing

### Import statements

In [ ]:
%%capture

import numpy as np
import tensorflow as tf

#mandatory for correct load and save of files
%cd /Users/paolobonomi/Work/Python/CapsNetwork

# for project class
import sys
sys.path.append("/Users/paolobonomi/Work/Python/CapsNetwork/src")

from setup import Setup # set up model and dataset
import perfu # performance function such as confusion matrix etc...
import printer

### Retrieve model and dataset

In [ ]:
print(Setup.d_k)

train_dataset_type = Setup.d_k[1]
test_dataset_type = Setup.d_k[0]
sould_be_trained=False
should_create_dataset=False
epochs=10

In [ ]:
setup = Setup(train_cfg=train_dataset_type, test_cfg=test_dataset_type, should_be_trained=sould_be_trained, 
                  should_create_dataset=should_create_dataset, epochs=epochs)

model = setup.get_model()
epochs = model.get_epochs()

X_train, y_train = setup.get_train_images()
X_test, y_test = setup.get_test_images()
no_train_images = setup.get_no_train_images()
no_test_images = setup.get_no_test_images()
dataset = setup.get_dataset()
testing = setup.get_testing()

### Perfomance data

In [ ]:
printer.pretty_experiment_overview(setup)

In [ ]:
should_create_matrix = False

In [ ]:
p_train = train_dataset_type.replace("_", " ")
p_test = test_dataset_type.replace("_", " ")

if should_create_matrix:
    confusionmatrix_test_0 = perfu.get_confusion_mat(model.predict, testing, "test")
    %store confusionmatrix_test_0
    
    confusionmatrix_train_0 = perfu.get_confusion_mat(model.predict, dataset, "train")
    %store confusionmatrix_train_0
else:
    %store -r confusionmatrix_train_0
    %store -r confusionmatrix_test_0 
    
acc_train = perfu.get_accuracy(perfu.normalize_matrix( confusionmatrix_train_0, no_train_images))
acc_test = perfu.get_accuracy(perfu.normalize_matrix( confusionmatrix_test_0, no_test_images))

#### Testing Confusion Table

In [ ]:
values, columns, index = perfu.get_confusion_table(confusionmatrix_test_0, no_test_images)
printer.print_confusion_tables(values, columns, index)

#### Confusion Matrix

In [ ]:
######## TRAIN

printer.print_matrix( perfu.normalize_matrix( confusionmatrix_train_0, no_train_images),
                        'Normalized on total images(%)', 
                        '{0} ({1} epochs)'.format(p_train, epochs), 
                        '{0} train'.format(p_train), 
                         no_train_images)

printer.print_matrix( perfu.normalize_matrix_on_row( confusionmatrix_train_0),
                        'Normalized on on tot images in row(%)', 
                        '{0} ({1} epochs)'.format(p_train, epochs), 
                        '{0} train'.format(p_train),  
                         no_train_images,
                         row_labels=printer.get_sum_row_matrix_label(confusionmatrix_test_0))

printer.print_matrix( perfu.normalize_matrix_on_columns( confusionmatrix_train_0),
                        'Normalized on tot images in column(%)', 
                        '{0} ({1} epochs)'.format(p_train, epochs), 
                        '{0} train'.format(p_train),  
                         no_train_images,
                         col_labels=printer.get_sum_colum_matrix_label(confusionmatrix_train_0))

####### TEST

printer.print_matrix( perfu.normalize_matrix( confusionmatrix_test_0, no_test_images),
                        'Normalized on total images(%)', 
                        '{0} ({1} epochs)'.format(p_train, epochs), 
                        '{0} test'.format(p_test),  
                         no_test_images)

printer.print_matrix( perfu.normalize_matrix_on_row( confusionmatrix_test_0),
                        'Normalized on on tot images in row(%)', 
                        '{0} ({1} epochs)'.format(p_train, epochs), 
                        '{0} test'.format(p_test),  
                         no_test_images,
                         row_labels=printer.get_sum_row_matrix_label(confusionmatrix_test_0))

printer.print_matrix( perfu.normalize_matrix_on_columns( confusionmatrix_test_0),
                        'Normalized on tot images in column(%)', 
                        '{0} ({1} epochs)'.format(p_train, epochs), 
                        '{0} test'.format(p_test),  
                         no_test_images,
                         col_labels=printer.get_sum_colum_matrix_label(confusionmatrix_test_0))

#### Accuracy

In [ ]:
printer.print_accuracy(acc_train, acc_test, no_train_images, no_test_images)

### Error cases

In [ ]:
off = 44
n = 110
idx, pred = perfu.get_error_index(model, X_test[off:off+n], y_test[off:off+n], off)

print(idx)
print(pred)

In [ ]:
for i in range(len(idx)):
    img = idx[i]
    printer.print_image_and_prediction(X_test[img], y_test[img], pred[i], 40 )


### Network Conv1/PrimaryCapsule Output

In [ ]:
printer.print_network(model, X_test[45], y_test[45], 40)

### Network Kernels

In [ ]:
printer.print_fixed_network_params(model)